In [ ]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="./src/configs/datasets"):
    cfg = compose(config_name="sasrec")

cfg

In [ ]:
val_config = cfg["train"]
val_config["name"] = "test"
val_config["instance_transforms"] = None
val_config = dict(val_config)
val_config.pop("_target_")

In [ ]:
from src.datasets import YambdaDataset

dataset = YambdaDataset(**val_config)

In [ ]:
import torch

cnt = 0
k = 10
seen_items = set()

for data in dataset:
    values, counts = torch.unique(data["seq"], return_counts=True)
    top_k_indices = torch.argsort(counts, descending=True)[:k]
    recs = values[top_k_indices]
    seen_items |= set(recs.cpu().numpy())
    if torch.any(recs == data["item"]):
        cnt += 1

print(f"HR@{k}: {cnt / len(dataset)}, COV@{k}: {len(seen_items) / dataset.n_items}")